 <--- The code outputs stastistical data after cleaning the files and converting it into tasks--->

In [27]:
# Import necessary packages

import glob
import re
import csv
import os
from operator import add
from pathlib import Path
import indicnlp
import codecs
from collections import defaultdict
import matplotlib.pyplot as plt

In [28]:
#Defining minimum/ maximum lengths for all languages

MIN_MAR = 4
MAX_MAR = 10

MIN_TEL =4
MAX_TEL =10

MIN_TAM = 3
MAX_TAM = 10

MIN_MAL = 3
MAX_MAL = 10

MIN_OTH =4
MAX_OTH =10

waste_len =0

In [29]:
# Get the min,max length for the language passed

def get_len(language):
    if language=="Marathi" or language == "Hindi":
        return MIN_MAR,MAX_MAR
    elif language=="Telugu":
        return MIN_TEL,MAX_TEL
    elif language =="Tamil":
        return MIN_TAM,MAX_TAM
    elif language =="Malayalam":
        return MIN_MAL,MAX_MAL
    else :
        return MIN_OTH,MAX_OTH


In [30]:
#Divide two numbers

def div(x,y):
    try:
        return x/y
    except ZeroDivisionError:
        return 0

In [31]:
# generate csv file from observed data

def write_list_to_file(guest_list, filename):
    with open(filename,'a') as outfile:
        wr = csv.writer(outfile, dialect='excel')
        wr.writerow(guest_list)

In [32]:
# Performing special characters check in the string passed

def checks(string):
    regex = '-'
    regex1 = '[A-Za-z]+'
    regex2 = '/'
    regex3 = '\/'
		
    unwanted_symbols = ['/', '-', '[', '\\', ']', '॥', '।' ,'<', '>', '”', '“', '*', '*']
    if string == "":
        return False
    for symbol in unwanted_symbols:
        if string.find(symbol) != -1:
            return False
        if re.match(regex, string):
            return False		
        if re.match(regex2, string):
            return False
        if re.match(regex3, string):
            return False
        if string.find("/") != -1:
            return False
        if string.find('-') != -1:
            return False
        if string.find('[') != -1:
            return False
        if string.find("\\") != -1:
            return False
        if string.find(']') != -1:
            return False
        if string.find('॥') != -1:
            return False
        if string.find('।') != -1:
            return False
        if string.find('<') != -1:
            return False
        if string.find('>') != -1:
            return False
        if string.find('”') != -1:
            return False
        if string.find('“') != -1:
            return False
        for ch in range(0,26):
            if string.find(chr(65+ch)) != -1 or string.find(chr(97+ch)) != -1:
                return False
        return True

In [33]:
# Applying filters on the passed sentences

def make_textfile_object(sentences, min_length, max_length, task_type_dum, task_type, language, path , seperator):
    f1 = open(path, "w+")
    waste_len=0
    filtered_sentences  = []
    filtered_sentences_specialchar = []
    filtered_sentences_length = []
    for dat in sentences:
        dat = dat.replace('\xc2\xa0', '')
        dat = dat.replace('\n', '')
        dat = dat.replace('  ', '')
        dat = dat.strip()
        if task_type_dum == 'quote':
             dat = dat.split('"')[1]
             dat = dat.strip()
        elif task_type_dum == 'quote2':
             dat = dat.split("'")[1]
             dat = dat.strip()
        elif task_type_dum == 'diag':
            dat = dat.split(':')[-1]
            dat = dat.strip()
        len_dat = len(dat.split(' '))
        
        if len_dat >= min_length and len_dat <= max_length and checks(dat)==True:
            filtered_sentences.append(dat)
            
    for sent in filtered_sentences:
        f1.write(sent)
        f1.write(seperator)
        #f1.write(u"।")
        f1.write('\n')

    return len(filtered_sentences)

In [34]:
# Check if the name is a language

def check_lang(lang):
    if lang =='Hindi' or lang=='Bengali' or lang=='Odia' or lang=='Tamil' or lang =='Telugu' or lang == 'Marathi' or lang == 'Malayalam' or lang == 'Punjabi' or lang=='Gujrati' or lang=='Kannada':
        return True
    else:
        return False

In [35]:
# Seperator for various languages

def seperator(language):
    if language =='Hindi' or language =='Bengali' or language=='Odia' or language =='Punjabi':
        return u"।"
    else: 
        return "."



In [36]:
match =0
my_path = '/home/ankita/CLAP/Data'
write_path = '/home/ankita/CLAP/Data_1/'
csv_path =   '/home/ankita/CLAP/Data/summary.csv'
regex = '[!|?|\.|\"|\'|\*|\-|\[|\]]'
regex_dot = r'[^\.!?]*[!\?\.]'
regex_quote = r'[\"][^\"]*[\"]'
regex_quote2 = r'[\'][^\']*[\']'
regex_colon = '[^\.!?:]*[:][^\.!?:]*'

In [37]:
#Extracting required data from the given path, source and language

def extract_data(my_path,language,source):
    
    # Create a path to write the files after filter
    
    filter_path = os.path.join(write_path, source, language)
    if not os.path.isdir(filter_path):
        os.makedirs(filter_path)
    
    #Initialise counters for each source and language
    
    total_tasks=0
    total_filtertasks =0
        
    #Iterate through all text files for the source and language
    
    for file in Path(my_path).glob('**/*.txt'):
        taskvalue =[]
        textFile_str = str(file)
        path_list = textFile_str.split(os.sep)
        min_length,max_length = get_len(language)
        textFile = open(textFile_str, 'rb')	
    
        data = textFile.read()
        data = data.decode('utf-8')
        data = re.sub('[\n0-9()]', ' ', data)
        data = re.sub(' +', ' ',data)
        data = re.sub('\n+', '', data)
        
        textFile_str = textFile_str.split('/')[-1].split('.')[0]
        filter_data_path = filter_path +"/"+textFile_str
        
        quote_data = re.findall(regex_quote, data)
        quote_sen_len  = make_textfile_object(quote_data, min_length, max_length, "quote", "Conversational", language,filter_data_path, seperator(language))
        data = re.sub(regex_quote, '' ,data)
        
        quote_data2 = re.findall(regex_quote2, data)
        quote2_sen_len = make_textfile_object(quote_data2, min_length, max_length, "quote2","Conversational", language,filter_data_path, seperator(language))
        data = re.sub(regex_quote2, '' ,data)
        
        diag_data = re.findall(regex_colon, data)
        diag_sen_len = make_textfile_object(diag_data, min_length, max_length, "diag","Dialouge", language, filter_data_path, seperator(language))
        data = re.sub(regex_colon, '' ,data)
        
        reg_data =data.split(seperator(language))
        reg_sen_len = make_textfile_object(reg_data, min_length, max_length, "regular","Regular", language, filter_data_path, seperator(language))
        
        total_data =len(quote_data)+len(quote_data2)+len(diag_data)+len(reg_data)
        total_sen_len =quote_sen_len+quote2_sen_len+diag_sen_len+reg_sen_len
        
        total_tasks +=total_data
        total_filtertasks +=total_sen_len
    
    return total_tasks,total_filtertasks

In [38]:
# Extracting data from all directory and dumping filtered data to a new directory following directory structure

Data_source = [f.path for f in os.scandir(my_path) if f.is_dir()]

for data_path in Data_source:
    
    source =data_path.split('/')[-1]
    lang_for_source = [f.path for f in os.scandir(data_path) if f.is_dir()]
    
    for lang_path in lang_for_source:
        
        language =lang_path.split('/')[-1]        
        if(check_lang(language) is False):
            continue
        total_tasks,total_filtertasks = extract_data(lang_path,language,source)
        data_list = [source,language,total_tasks,total_filtertasks]
        write_list_to_file(data_list,csv_path)
        
        

<---END OF CODE , further processing in  analyser part 2--->